<a href="https://colab.research.google.com/github/ykitaguchi77/GO_AI_project/blob/main/Interference_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Interference using pretrained models**

In [1]:
import torch
from IPython.display import Image, clear_output
import os
import shutil

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#GDriveをマウント
from google.colab import drive
drive.mount('/content/drive')

Setup complete. Using torch 1.12.1+cu113 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)
Mounted at /content/drive


In [2]:
image_folder_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_newPatient_250px"
model_path = "/content/drive/MyDrive/Deep_learning/666mai_dataset/5-fold_20221019/Models/moblenetv3_large_100.pth"

In [3]:
# specify the image paths
image_list = [image_folder_path +"/スライド"+str(i+1)+".jpeg" for i in range(len(os.listdir(image_folder_path)))]
label_list = [1]*len(image_list)

In [ ]:
# show sample images
import matplotlib.pyplot as plt
import cv2
import os

root = image_folder_path #画像があるフォルダ。適宜変えてください
lsdir = os.listdir(root)

imgs = []
for l in lsdir:
    target = os.path.join(root,l)
    img = cv2.imread(target)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #pyplotで表示するために色変換
    imgs.append(img)

shownumber = 6 #画像を並べる数
showaxis = 1

while(showaxis*showaxis < shownumber):
    showaxis += 1

cnt = 0
while(1):
    limit = 6
    if cnt >= limit:
       break
    fig,axs = plt.subplots(showaxis,showaxis, figsize=(16.0, 12.0))
    ar = axs.ravel()
    for i in range(showaxis*showaxis):
        ar[i].axis('off')
        if i < shownumber:
            ar[i].imshow(imgs[cnt])
            cnt += 1
    plt.show()

###**Load model**

In [5]:
##########################
# Load model 
##########################
!pip install --quiet timm
import timm
import torch.nn as nn

model_ft = timm.create_model('mobilenetv3_large_100', pretrained=True)
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, 2)

#ネットワークの読み込み
model_ft.load_state_dict(torch.load(model_path))

     |████████████████████████████████| 548 kB 5.1 MB/s 
     |████████████████████████████████| 163 kB 70.2 MB/s 


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mobilenetv3_large_100_ra-f55367f5.pth" to /root/.cache/torch/hub/checkpoints/mobilenetv3_large_100_ra-f55367f5.pth


<All keys matched successfully>

In [6]:
from torchvision import datasets, models, transforms
from PIL import *
import pandas as pd

img_transforms = transforms.Compose([
                #Expand2square((0,0,0)),
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

def image_loader(image_path):
    """load image, returns cuda tensor"""
    image = Image.open(image_path)
    image = img_transforms(image).float()
    image = image.unsqueeze(0) 
    return image.to(device)

def my_round(val, digit=0):
    p = 10 ** digit
    return (val * p * 2 + 1) // 2 / p

def interference(image_list):
    image_tensor = image_loader(path)

    model_ft.to(device)
    model_ft.eval()
    output = model_ft(image_tensor)
    _, pred = torch.max(output, 1) 
    pred = pred[0].to('cpu').detach().numpy().copy().tolist() 

    prob = nn.Softmax(dim=1)(output) #calculate probalility
    prob = prob[0][1].cpu().detach().numpy().copy().tolist() #probalility of being positive
    prob = my_round(prob, 3)

    return pred, prob



pred_list, prob_list = [], []
for idx, path in enumerate(image_list, 1):
    pred, prob = interference(path) 
    pred_list.append(pred)
    prob_list.append(prob)
print(label_list)
print(pred_list)
print(prob_list)

df = pd.DataFrame(index=[], columns=[])
df["label"] = label_list
df["pred"] = pred_list
df["prob"] = prob_list
df

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0.623, 0.388, 0.936, 0.966, 0.838, 0.96, 0.806, 0.853, 0.964, 0.982, 0.542, 0.873, 0.83, 0.653, 0.95, 0.99, 0.165, 0.706, 0.499, 0.851, 0.989, 0.707, 0.315, 0.968, 0.557, 0.582, 0.989, 0.972, 0.112, 0.654, 0.964, 0.729, 0.985, 0.423, 0.094, 0.992, 0.907, 0.959, 0.842, 0.997, 0.855, 0.99, 0.758, 0.887, 0.622, 0.941]


,label,pred,prob
0,1,1,0.623
1,1,0,0.388
2,1,1,0.936
3,1,1,0.966
4,1,1,0.838
5,1,1,0.960
6,1,1,0.806
7,1,1,0.853
8,1,1,0.964
9,1,1,0.982


#**Output CoreML**

In [7]:
###########################
# Output as CoreML 飛ばして下さい
###########################

import torch
import torchvision
!pip install --quiet coremltools
import coremltools as ct

# Load a pre-trained version of MobileNetV3
class TorchClassificationModel(nn.Module):
    def __init__(self):
        super(TorchClassificationModel, self).__init__()
        self.layers = nn.Sequential(
            model_ft,
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        return self.layers(x)


# Set the model in evaluation mode
torch_model = TorchClassificationModel().eval()
torch_model = torch_model.to("cpu")


# Trace with random data
example_input = torch.rand(1, 3, 224, 224) # after test, will get 'size mismatch' error message with size 256x256
traced_model = torch.jit.trace(torch_model, example_input)


# Download class labels (from a separate file)
#import urllib
#label_url = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
#class_labels = urllib.request.urlopen(label_url).read().decode("utf-8").splitlines()
class_labels = ["cont", "grav"]



# Convert to Core ML using the Unified Conversion API
mlmodel = ct.convert(
    traced_model,
    inputs=[ct.ImageType(name="input_1", shape=example_input.shape)], #name "input_1" is used in 'quickstart'
    classifier_config = ct.ClassifierConfig(class_labels) # provide only if step 2 was performed
)

# Save model
mlmodel.save("/content/gravcont_mobilenetv3.mlmodel")


     |████████████████████████████████| 1.5 MB 5.2 MB/s 


Running MIL Common passes:   0%|          | 0/38 [00:00<?, ? passes/s]/usr/local/lib/python3.7/dist-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '879', of the source model, has been renamed to 'var_879' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 665/665 [00:00<00:00, 1878.64 ops/s]


In [13]:
# This script can be used in Mac only

##https://gist.github.com/ozgurshn/85cf74558d82c831827e12f015f752a1
##https://github.com/apple/coremltools/blob/master/examples/APIExamples.md
import coremltools
import numpy as np
import PIL.Image

# load a model whose input type is "Image"
model = coremltools.models.MLModel('/content/gravcont_mobilenetv3.mlmodel')

Height = 224  # use the correct input image height
Width = 224  # use the correct input image width


# Scenario 1: load an image from disk
def load_image(path, resize_to=None):
    # resize_to: (Width, Height)
    img = PIL.Image.open(path)
    if resize_to is not None:
        img = img.resize(resize_to, PIL.Image.ANTIALIAS)
    img_np = np.array(img).astype(np.float32)
    return img_np, img


# load the image and resize using PIL utilities
_, img = load_image('/content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_newPatient_250px/スライド1.jpeg', resize_to=(Width, Height))
out_dict = model.predict({'image': img})

# Scenario 2: load an image from a numpy array
shape = (Height, Width, 3)  # height x width x RGB
data = np.zeros(shape, dtype=np.uint8)
# manipulate numpy data
pil_img = PIL.Image.fromarray(data)
out_dict = model.predict({'image': pil_img})

Exception: ignored